## DCOPF
- https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/06-Optimal-Power-Flow.ipynb

In [25]:
using JuMP, HiGHS, CSV, DataFrames, PrettyTables, VegaLite

In [26]:
gen = CSV.read("gen.csv", DataFrame);
gencost = CSV.read("gencost.csv", DataFrame);
branch = CSV.read("branch.csv", DataFrame);
bus = CSV.read("bus.csv", DataFrame);

In [27]:
for f in [gen, gencost, branch, bus]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gen.id = 1:nrow(gen);
gencost.id = 1:nrow(gencost);

# create line ids 
branch.id = 1:nrow(branch);
# add set of rows for reverse direction with same parameters
branch2 = copy(branch)
branch2.f = branch2.fbus
branch2.fbus = branch.tbus
branch2.tbus = branch2.f
branch2 = branch2[:,names(branch)]
append!(branch,branch2)

# Calculate the susceptance of each line, on the assumption that
# reactance is >> resistance, such that we can approximate 
# resistance as = 0 and treat susceptance as the simple  
# reciprocal of reactance (x). 
# See https://en.wikipedia.org/wiki/Susceptance#Relationship_to_reactance
branch.sus = 1 ./ branch.x 

# Here are the buses:
display(bus)
display(gen)
display(branch)
display(gencost)

Row,bus_i,type,pd,qd,gs,bs,area,vm,va,basekv,zone,vmax,vmin
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64
1,1,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
2,2,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
3,3,1,600,98.61,0,0,1,1,0,230,1,1.1,0.9


Row,bus,pg,qg,qmax,qmin,vg,mbase,status,pmax,pmin,pc1,pc2,qc1min,qc1max,qc2min,qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf,id
,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,40,0,30.0,-30.0,1,100,1,1000,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,170,0,127.5,-127.5,1,100,1,1000,0,0,0,0,0,0,0,0,0,0,0,0,2


Row,fbus,tbus,r,x,b,ratea,rateb,ratec,ratio,angle,status,angmin,angmax,id,sus
,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,1,3,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,1,35.5872
2,1,2,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,2,35.5872
3,2,3,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,3,35.5872
4,3,1,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,1,35.5872
5,2,1,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,2,35.5872
6,3,2,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,3,35.5872


Row,model,startup,shutdown,n,x1,y1,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2,0,0,2,50,0,1
2,2,0,0,2,100,0,2


In [28]:
println(gen.id)
println(bus.bus_i)

[1, 2]
[1, 2, 3]


In [29]:
model = Model(HiGHS.Optimizer);

# Define Sets
G = gen.id;    # set of generators
N = bus.bus_i; # set of buses

# Define per unit base units for the system
base_MVA = gen.mbase[1];

# Decision Variables
@variables(model, begin
    GEN[G] >= 0
    THETA[N]
    FLOW[N, N]
end);

# Create Slack bus
fix(THETA[1], 0);

# objective_function
@objective(model, Min, sum( gencost[g, :x1] * GEN[g] for g in G));

# Power Balance
@constraint(model, cBalance[i in N],
    sum(GEN[g] for g in gen[gen.bus .== i, :id]) - bus[bus.bus_i .== i, :pd][1]
    ==
    sum(FLOW[i,j] for j in branch[branch.fbus .== i, :tbus])
);

# Max Generation
@constraint(model, cMaxGen[g in G],
    GEN[g] <= gen[g, :pmax]
)

# 
@constraint(model, cLineFlow[l in 1:nrow(branch)],
    FLOW[branch[l, :fbus], branch[l, :tbus]] # sus: susceptance
    == 
    base_MVA*branch[l, :sus] *(THETA[branch[l, :fbus]] - THETA[branch[l, :tbus]])
);

# Max Line Flow
@constraint(model, cLineLimits[l in 1:nrow(branch)],
    FLOW[branch[l, :fbus], branch[l, :tbus]] <= branch[l, :ratea]
);

# Solve statement
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
8 rows, 9 cols, 20 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-17); columns 0(-14); elements 0(-34) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.0000000000e+04
HiGHS run time      :          0.00


In [31]:
generation = DataFrame(id=gen.id,
                    node = gen.bus, 
                    gen = value.(GEN).data
)

display(generation)
println("cost: $(objective_value(model))")
display(value.(FLOW).data)

Row,id,node,gen
,Int64,Int64,Float64
1,1,1,600.0
2,2,2,0.0


cost: 30000.0


3×3 Matrix{Float64}:
    0.0   200.0  400.0
 -200.0     0.0  200.0
 -400.0  -200.0    0.0